<a href="https://colab.research.google.com/github/mille055/Computer-Vision-for-Elementary-Education/blob/main/Train_classifier_from_repo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone 'https://github.com/iqra0908/Computer-Vision-for-Elementary-Education.git/'

Cloning into 'Computer-Vision-for-Elementary-Education'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 169 (delta 44), reused 26 (delta 5), pack-reused 62
Receiving objects: 100% (169/169), 38.24 MiB | 23.22 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [3]:
%cd 'Computer-Vision-for-Elementary-Education'

/content/Computer-Vision-for-Elementary-Education


In [4]:
!python train_transfer_classifier.py

torch:  1.13 ; cuda:  cu116
Traceback (most recent call last):
  File "train_transfer_classifier.py", line 67, in <module>
    train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'),
  File "/usr/local/lib/python3.8/dist-packages/torchvision/datasets/folder.py", line 309, in __init__
    super().__init__(
  File "/usr/local/lib/python3.8/dist-packages/torchvision/datasets/folder.py", line 144, in __init__
    classes, class_to_idx = self.find_classes(self.root)
  File "/usr/local/lib/python3.8/dist-packages/torchvision/datasets/folder.py", line 218, in find_classes
    return find_classes(directory)
  File "/usr/local/lib/python3.8/dist-packages/torchvision/datasets/folder.py", line 40, in find_classes
    classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
FileNotFoundError: [Errno 2] No such file or directory: 'data/toys_data/train'


In [13]:
!ls

Computer-Vision-for-Elementary-Education  sample_data


In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
##### Train model using transfer learning on ResNet50 architecture 
##### Saves model.pkl with name containing current date
##### Assumes dataset structure of images in directory with folder names = the classes
##### Heavily borrows from colab notebook from the class repo

import os
import urllib.request
import zipfile
import copy
import time
import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms
import torchvision
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchsummary import summary

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from datetime import datetime

# Some parameters being used
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

model_filename = 'resnet50classifier'+datetime.now().strftime("%Y%m%d")+'.pkl'
#data_dir = 'data/toys_data'
data_dir = '/content/gdrive/MyDrive/AIPI540/'
batch_size = 8
num_epochs = 25

# Set up transformations for training, validation, and test data
# For training data we will do resize to 224 * 224, randomized horizontal flipping, rotation, lighting effects, and normalization
# For test set we will do only center cropping to get to 224 * 224 and normalization

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(257),
		    transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
		    transforms.RandomVerticalFlip(),
	      transforms.ColorJitter(brightness=(0.0,1.5), contrast=(1)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Create Datasets for training and validation sets
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'),
                                          data_transforms['train'])
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'),
                                          data_transforms['val'])
#test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test'])


# Create DataLoaders for training and validation sets

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                             shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,
                                             shuffle=False, num_workers=2)

# Set up dict for dataloaders
dataloaders = {'train':train_loader,'val':val_loader}

# Store size of training and validation sets
dataset_sizes = {'train':len(train_dataset),'val':len(val_dataset)}

# Get class names associated with labels
class_names = train_dataset.classes
num_classes = len(class_names)
print(f'there are {num_classes} classes including {class_names}')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device is {device}')

# Instantiate pre-trained resnet
net = torchvision.models.resnet50(pretrained=True)
# Shut off autograd for all layers to freeze model so the layer weights are not trained
for param in net.parameters():
    param.requires_grad = False

# Display a summary of the layers of the model and output shape after each layer
#summary(net,(images.shape[1:]),batch_size=batch_size,device="cpu")


# Get the number of inputs to final Linear layer
num_ftrs = net.fc.in_features
# Replace final Linear layer with a new Linear with the same number of inputs but just num_classes outputs,

net.fc = nn.Linear(num_ftrs, num_classes)

# Cross entropy loss combines softmax and nn.NLLLoss() in one single class.
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

def train_model(model, criterion, optimizer, dataloaders, scheduler, device, num_epochs=num_epochs):
    model = model.to(device) # Send model to GPU if available
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Get the input images and labels, and send to GPU if available
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the weight gradients
                optimizer.zero_grad()

                # Forward pass to get outputs and calculate loss
                # Track gradient only for training data
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backpropagation to get the gradients with respect to each weight
                    # Only if in train
                    if phase == 'train':
                        loss.backward()
                        # Update the weights
                        optimizer.step()

                # Convert loss into a scalar and add it to running_loss
                running_loss += loss.item() * inputs.size(0)
                # Track number of correct predictions
                running_corrects += torch.sum(preds == labels.data)

            # Step along learning rate scheduler when in train
            if phase == 'train':
                scheduler.step()

            # Calculate and display average loss and accuracy for the epoch
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # If model performs better on val set, save weights as the best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:3f}'.format(best_acc))

    # Load the weights from best model
    model.load_state_dict(best_model_wts)

    return model


# Learning rate scheduler - decay LR by a factor of 0.1 every 7 epochs
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Train the model
net = train_model(net, criterion, optimizer, dataloaders, lr_scheduler, device, num_epochs=25)

# Save the trained model as a pkl file
net.to_pickle(model_filename)
print('Saving model to {model_filename}')



torch:  1.13 ; cuda:  cu116
there are 8 classes including ['cow', 'dog', 'goat', 'hen', 'horse', 'llama', 'pig', 'sheep']
Device is {device}


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Epoch 0/24
----------
train Loss: 2.0722 Acc: 0.1329
val Loss: 2.0664 Acc: 0.0968

Epoch 1/24
----------
train Loss: 1.9744 Acc: 0.2517
val Loss: 1.8305 Acc: 0.4516

Epoch 2/24
----------
train Loss: 1.7574 Acc: 0.4056
val Loss: 1.7433 Acc: 0.4516

Epoch 3/24
----------
train Loss: 1.6822 Acc: 0.4476
val Loss: 1.4766 Acc: 0.5806

Epoch 4/24
----------
train Loss: 1.5507 Acc: 0.5035
val Loss: 1.2573 Acc: 0.6129

Epoch 5/24
----------
train Loss: 1.4838 Acc: 0.4685
val Loss: 1.2659 Acc: 0.6452

Epoch 6/24
----------
train Loss: 1.2888 Acc: 0.7063
val Loss: 1.1145 Acc: 0.7419

Epoch 7/24
----------
train Loss: 1.2185 Acc: 0.7203
val Loss: 1.1126 Acc: 0.7097

Epoch 8/24
----------
train Loss: 1.1828 Acc: 0.7343
val Loss: 1.1108 Acc: 0.7419

Epoch 9/24
----------
train Loss: 1.2486 Acc: 0.6573
val Loss: 1.0615 Acc: 0.7097

Epoch 10/24
----------
train Loss: 1.1501 Acc: 0.7692
val Loss: 1.0255 Acc: 0.7742

Epoch 11/24
----------
train Loss: 1.1274 Acc: 0.7762
val Loss: 0.9940 Acc: 0.8387

Ep

AttributeError: ignored

In [18]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [25]:
torch.save(net.state_dict(), '/content/resnet_transfer_model0207')